In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Database credentials 
username = "aacuser"
password = "guest123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# Remove the MongoDB _id column to prevent compatibility issues in Dash tables
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encode the logo image in Base64 format for embedding in Dash
with open('Grazioso Salvare Logo.png', 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Layout of the dashboard
app.layout = html.Div([
    
    # Branding and Title Section
    html.Div([
        html.A(
            html.Img(
                src=f"data:image/png;base64,{encoded_image}",
                style={'height': '200px', 'width':'auto'}
                ),
                href="https://www.snhu.edu",
                target="_blank"
        ),
        html.H1("Grazioso Salvare Dashboard", style={'textAlign': 'center'}),
        html.H3("Created by Matthew Guarino - 2/20/2025", style={'textAlign': 'center'})
    ], style={'textAlign': 'center'}),
    
    # Filter Controls
html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster/Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='All',
            labelStyle={'display': 'inline-block'}
        )
]),

# Data Table for displaying filtered dog data
dash_table.DataTable(id='datatable-id', 
    columns=[
        {'name': 'name', 'id': 'name', 'type': 'text'},
        {'name': 'breed', 'id': 'breed', 'type': 'text'},
        {'name': 'age_upon_outcome_in_weeks', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric'},
        {'name': 'sex_upon_outcome', 'id': 'sex_upon_outcome', 'type': 'text'},
        {'name': 'location_lat', 'id': 'location_lat', 'type': 'numeric'},
        {'name': 'location_long', 'id': 'location_long', 'type': 'numeric'}
    ], 
    page_size=10, 
    style_table={'overflowX': 'auto'}, 
    row_selectable='single'),

    # Side-by-side layout for Graph and Map
html.Div([
        html.Div(id='graph-id', style={'width': '50%', 'display': 'inline-block'}),
        html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block'})
    ], style={'display': 'flex'})
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on selected filter
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {}
    
    if filter_type == 'Water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}  
        }

    elif filter_type == 'Mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}  
        }

    elif filter_type == 'Disaster':
        query = {
            "breed": {"$in": ["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300} 
        }
    elif filter_type in ['All', 'Reset']: 
        query = {
            "breed": {"$in":[
                "Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland Mix", "German Shepherd", "Alaskan Malamute", 
                "Old English Sheepdog", "Siberian Husky", "Rottweiler", "Doberman Pinsch", "Golden Retriever", "Bloodhound",
                "Newfoundland/Labrador Retriever", "Labrador Retriever/Newfoundland"
            ]}
        }
        
# Fetch filtered data from MongoDB
    filtered_data = db.read(query)
    df_filtered = pd.DataFrame.from_records(filtered_data)
    
    # Clean up DataFrame before returning
    if '_id' in df_filtered:
        df_filtered.drop(columns=['_id'], inplace=True)
        
    # Ensure numeric values for age
    if 'age_upon_outcome_in_weeks' in df_filtered:
        df_filtered['age_upon_outcome_in_weeks'] = pd.to_numeric(df_filtered['age_upon_outcome_in_weeks'], errors='coerce')

    return df_filtered.to_dict('records')

# Callback to update the pie chart
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return [html.P("No data available", style={'textAlign': 'center', 'color': 'red'})]
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff:
        return [html.P("Breed data missing", style={'textAlign': 'center', 'color': 'red'})]
    fig = px.pie(dff, names='breed', title='Preferred Animal Breeds')
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns: 
        return []
    return [{'if': {'column_id': i}, 'backgroundColor': '#D2F3FF'} for i in selected_columns]

# Callback to update the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    
    # Updates the map based on the selected dog.
    
    if not viewData or not index:
        return [html.P("No data available", style={'textAlign': 'center', 'color': 'red'})]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index and index[0] < len(dff) else 0
    lat = dff.iloc[row].get("location_lat", 30.75)
    lon = dff.iloc[row].get("location_long", -97.48)

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(dff.iloc[row].get("breed", "Unknown")),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row].get("name", "Unknown"))
            ])
        ])
    ])]

app.run_server(mode='external', debug=True)

C:\dash_env\venv\Lib\site-packages\dash\dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


Dash app running on http://127.0.0.1:8050/
